# 2. Pre-processamiento: Volatilidad y deflacion de precios

In [13]:
# manipulacion archivos
import os

# manipulacion datos
import pandas as pd

In [14]:
data_market = pd.read_pickle(os.path.join("1_data","data_market.pkl"))
data_market.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34078 entries, 2072 to 56249
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   coin_id        34078 non-null  object 
 1   prices         34078 non-null  float64
 2   year           34078 non-null  int32  
 3   month          34078 non-null  int32  
 4   day            34078 non-null  int32  
 5   market_caps    34078 non-null  float64
 6   total_volumes  34078 non-null  float64
 7   category_id    34078 non-null  object 
 8   categories     34078 non-null  object 
dtypes: float64(3), int32(3), object(3)
memory usage: 2.2+ MB


## 2.1. Volatilidad relativa en el corto plazo: precios corrientes

In [15]:
data_market_prcvoltlty_group_mean = pd.DataFrame(
    data_market.groupby(['category_id','coin_id','year'])['prices'].std().reset_index()
    .groupby(['category_id','year'])['prices'].mean()
    ).rename(columns={'prices':'sd_group_mean'}).reset_index()
data_market_prcvoltlty_group_mean

,category_id,year,sd_group_mean
0,layer-1,2019,255.734510
1,layer-1,2020,402.039620
2,layer-1,2021,1023.298805
3,layer-1,2022,1018.478820
4,layer-1,2023,347.729380
5,stablecoins,2019,2.822968
6,stablecoins,2020,14.540918
7,stablecoins,2021,5.314946
8,stablecoins,2022,9.663229
9,stablecoins,2023,6.194019


In [16]:
data_market_prcvoltlty_coin = pd.DataFrame(
    data_market.groupby(['category_id','coin_id','year'])['prices'].std()
    ).rename(columns={'prices':'sd_coin'}).reset_index()
data_market_prcvoltlty_crrnt = data_market_prcvoltlty_coin.merge(
    data_market_prcvoltlty_group_mean,
    how='left')
data_market_prcvoltlty_crrnt['vltlty_ratio'] = data_market_prcvoltlty_crrnt['sd_coin'] / data_market_prcvoltlty_crrnt['sd_group_mean']
data_market_prcvoltlty_crrnt['vltlty_rank']=data_market_prcvoltlty_crrnt.groupby(['category_id','year'])[
    'vltlty_ratio'].rank(method='dense', ascending=False)
data_market_prcvoltlty_crrnt.sort_values(
    by=['category_id','year','vltlty_rank'],
    kind='stable',
    inplace=True)
data_market_prcvoltlty_crrnt.to_pickle(os.path.join("2_pipeline","data_market_prcvoltlty_crrnt.pkl"))
data_market_prcvoltlty_crrnt.head(11)

,category_id,coin_id,year,sd_coin,sd_group_mean,vltlty_ratio,vltlty_rank
10,layer-1,bitcoin,2019,2655.712480,255.73451,10.384646,1.0
15,layer-1,bitcoin-cash,2019,96.242064,255.73451,0.376336,2.0
35,layer-1,ethereum,2019,50.775028,255.73451,0.198546,3.0
5,layer-1,binancecoin,2019,8.374184,255.73451,0.032746,4.0
25,layer-1,cosmos,2019,1.132393,255.73451,0.004428,5.0
0,layer-1,algorand,2019,0.411165,255.73451,0.001608,6.0
50,layer-1,tezos,2019,0.372936,255.73451,0.001458,7.0
30,layer-1,crypto-com-chain,2019,0.021568,255.73451,0.000084,8.0
20,layer-1,cardano,2019,0.018532,255.73451,0.000072,9.0
45,layer-1,hedera-hashgraph,2019,0.012550,255.73451,0.000049,10.0


## 2.2. Volatilidad en el mediano plazo: precios constantes
En esta seccion, deflacionamos los precios corrientes en precios constantantes.

Para el caso del dolar, el proceso se sencillo: deflacionamos los precios corrientes usando indices del precio al consumidor de EEUU.

Sin embargo, para el caso de monedas latinoamericanas el proceso es un poco diferentes. Primero, tenemos que convertir los precios en USD a la moneda local. Una vez ya estan convertido, porecedemo a deflacionarlos precios corrientes en moneda local usando indices del precio del pais de origen.

### 2.2.1. Transformacion a precios constantes en moneda local

In [17]:
data_deflators = pd.read_pickle(os.path.join("1_data","data_deflators.pkl"))
data_deflators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   code_pais       40 non-null     object 
 1   year            40 non-null     int64  
 2   gdpdfltr_2015   40 non-null     float64
 3   pppfactor_2015  40 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.4+ KB


In [21]:
data_market_constant = data_market.merge(
    data_deflators,
    on='year'
)
data_market_constant.info()
data_market_constant.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144875 entries, 0 to 144874
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   coin_id         144875 non-null  object 
 1   prices          144875 non-null  float64
 2   year            144875 non-null  int32  
 3   month           144875 non-null  int32  
 4   day             144875 non-null  int32  
 5   market_caps     144875 non-null  float64
 6   total_volumes   144875 non-null  float64
 7   category_id     144875 non-null  object 
 8   categories      144875 non-null  object 
 9   code_pais       144875 non-null  object 
 10  gdpdfltr_2015   144875 non-null  float64
 11  pppfactor_2015  144875 non-null  float64
dtypes: float64(5), int32(3), object(4)
memory usage: 11.6+ MB


,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories,code_pais,gdpdfltr_2015,pppfactor_2015
0,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",ARG,78.354031,0.685846
1,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",BRA,102.945851,0.597797
2,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",COL,100.505932,0.465566
3,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",MEX,101.312482,0.525458
4,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",USA,107.285879,1.000000
5,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",ARG,78.354031,0.685846
6,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",BRA,102.945851,0.597797
7,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",COL,100.505932,0.465566
8,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",MEX,101.312482,0.525458
9,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",USA,107.285879,1.000000


In [22]:
data_market_constant['prices_cnstnt2015'] = data_market_constant['prices'] * (100/data_market_constant['gdpdfltr_2015'])
data_market_constant['prices_ppp2015']=data_market_constant['prices_cnstnt2015']/data_market_constant['pppfactor_2015']
data_market_constant.head()

,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories,code_pais,gdpdfltr_2015,pppfactor_2015,prices_cnstnt2015,prices_ppp2015
0,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",ARG,78.354031,0.685846,4712.624899,6871.263049
1,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",BRA,102.945851,0.597797,3586.867784,6000.141521
2,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",COL,100.505932,0.465566,3673.943900,7891.355132
3,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",MEX,101.312482,0.525458,3644.695576,6936.229175
4,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]",USA,107.285879,1.000000,3441.768494,3441.768494


### 2.2.2. Volatilidad en precios constantes y moneda local

In [23]:
data_market_constant_prcvoltlty_group_mean = pd.DataFrame(
    data_market_constant.groupby(['category_id','code_pais','coin_id','year'])['prices_ppp2015'].std().reset_index()
    .groupby(['category_id','code_pais','year'])['prices_ppp2015'].mean()
    ).rename(columns={'prices_ppp2015':'sd_group_pais_mean'}).reset_index()
data_market_constant_prcvoltlty_group_mean.head(8)

,category_id,code_pais,year,sd_group_pais_mean
0,layer-1,ARG,2019,475.884650
1,layer-1,ARG,2020,782.470174
2,layer-1,ARG,2021,1739.810852
3,layer-1,ARG,2022,1403.234830
4,layer-1,BRA,2019,415.553185
5,layer-1,BRA,2020,801.906317
6,layer-1,BRA,2021,1917.492417
7,layer-1,BRA,2022,1687.184809


In [24]:
data_market_constant_prcvoltlty_group_median = pd.DataFrame(
    data_market_constant.groupby(['category_id','code_pais','coin_id','year'])['prices_ppp2015'].std().reset_index()
    .groupby(['category_id','code_pais','year'])['prices_ppp2015'].median()
    ).rename(columns={'prices_ppp2015':'sd_group_pais_median'}).reset_index()
data_market_constant_prcvoltlty_group_median.head(8)

,category_id,code_pais,year,sd_group_pais_median
0,layer-1,ARG,2019,0.765119
1,layer-1,ARG,2020,1.204551
2,layer-1,ARG,2021,2.542353
3,layer-1,ARG,2022,1.455064
4,layer-1,BRA,2019,0.668119
5,layer-1,BRA,2020,1.234471
6,layer-1,BRA,2021,2.801995
7,layer-1,BRA,2022,1.749502


In [25]:
data_market_constant_prcvoltlty_coin = pd.DataFrame(
    data_market_constant.groupby(['category_id','code_pais','coin_id','year'])['prices_ppp2015'].std()
    ).rename(columns={'prices_ppp2015':'sd_pais_coin'}).reset_index()
data_market_prcvoltlty_constant = data_market_constant_prcvoltlty_coin.merge(
    data_market_constant_prcvoltlty_group_mean,
    how='left').merge(
        data_market_constant_prcvoltlty_group_median,
        how='left')
data_market_prcvoltlty_constant.to_pickle(os.path.join("2_pipeline","data_market_prcvoltlty_constant.pkl"))
data_market_prcvoltlty_constant.head(11)

,category_id,code_pais,coin_id,year,sd_pais_coin,sd_group_pais_mean,sd_group_pais_median
0,layer-1,ARG,algorand,2019,0.765119,475.884650,0.765119
1,layer-1,ARG,algorand,2020,0.193741,782.470174,1.204551
2,layer-1,ARG,algorand,2021,0.758258,1739.810852,2.542353
3,layer-1,ARG,algorand,2022,0.463004,1403.234830,1.455064
4,layer-1,ARG,binancecoin,2019,15.583135,475.884650,0.765119
5,layer-1,ARG,binancecoin,2020,12.100110,782.470174,1.204551
6,layer-1,ARG,binancecoin,2021,289.322226,1739.810852,2.542353
7,layer-1,ARG,binancecoin,2022,99.495651,1403.234830,1.455064
8,layer-1,ARG,bitcoin,2019,4941.893849,475.884650,0.765119
9,layer-1,ARG,bitcoin,2020,8201.913436,782.470174,1.204551
